In [58]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("C:/Users/573380/Downloads/machine-learning-nd-master/machine-learning-nd-master/p2-student-intervention/student-data.csv")
print("Student data read successfully!")

Student data read successfully!


In [6]:
#describe will provide stats for only numeric values
student_data.describe()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,Walc,health,absences
count,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000,395.000000
mean,16.696203,2.749367,2.521519,1.448101,2.035443,0.334177,3.944304,3.235443,3.108861,1.481013,2.291139,3.554430,5.708861
std,1.276043,1.094735,1.088201,0.697505,0.839240,0.743651,0.896659,0.998862,1.113278,0.890741,1.287897,1.390303,8.003096
min,15.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,16.000000,2.000000,2.000000,1.000000,1.000000,0.000000,4.000000,3.000000,2.000000,1.000000,1.000000,3.000000,0.000000
50%,17.000000,3.000000,2.000000,1.000000,2.000000,0.000000,4.000000,3.000000,3.000000,1.000000,2.000000,4.000000,4.000000
75%,18.000000,4.000000,3.000000,2.000000,2.000000,0.000000,5.000000,4.000000,4.000000,2.000000,3.000000,5.000000,8.000000
max,22.000000,4.000000,4.000000,4.000000,4.000000,3.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,75.000000


In [7]:
student_data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,passed
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,no,4,3,4,1,1,3,6,no
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,no,5,3,3,1,1,3,4,no
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,no,4,3,2,2,3,3,10,yes
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,3,2,2,1,1,5,2,yes
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,no,4,3,2,1,2,5,4,yes


# Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
The total number of students, n_students.
The total number of features for each student, n_features.
The number of those students who passed, n_passed.
The number of those students who failed, n_failed.
The graduation rate of the class, grad_rate, in percent (%).

In [33]:
#Get the basic details
#The total number of students, n_students
n_students=len(student_data)

#The total number of features for each student, n_features.
n_features=len(student_data.iloc[0]) - 1
print(n_features)

#The number of those students who passed, n_passed.
n_passed=len(student_data[student_data['passed']=="yes"])

#The number of those students who failed, n_failed.
n_failed=len(student_data[student_data['passed']=="no"])

#The graduation rate of the class, grad_rate, in percent (%)
grad_rate=100*(n_passed/n_students)

# Print the results
print("Total number of students （number of datapoints): {}".format(n_students))
print("Number of features: {}".format(n_features))
print("Number of students who passed (graduates): {}".format(n_passed))
print("Number of students who failed (non-graduates): {}".format(n_failed))
print("Graduation rate of the class: {:.2f}%".format(grad_rate))

30
Total number of students （number of datapoints): 395
Number of features: 30
Number of students who passed (graduates): 265
Number of students who failed (non-graduates): 130
Graduation rate of the class: 67.09%


In [43]:
# Experiment to see if `failures` are a good predictor of `passed`
pp,pf,fp,ff = 0, 0, 0, 0
student_data[['failures','passed']]
for i in range(len(student_data)):
    if student_data.iloc[i]['failures'] > 0:
        if student_data.iloc[i]['passed'] == 'yes':
            fp+=1
        else:
            ff+=1
    else: 
        if student_data.iloc[i]['passed'] == 'no':
            pf+=1
        else: 
            pp+=1
            
print("pp: ", pp, "pf: ", pf, "fp: ", fp, "ff: ", ff)

pp:  234 pf:  78 fp:  31 ff:  52


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.
Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.
Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [46]:
#Extract feature columns
feature_cols=list(student_data.columns[:-1])

#Extract target columns
target_col=student_data.columns[-1]

# Show the list of columns
print("Feature columns:\n{}".format(feature_cols))
print("\nTarget column: {}".format(target_col))

X_all = student_data[feature_cols]
Y_all = student_data[target_cols]

# Show the feature information by printing the first five rows
print("\nFeature values:")
print(X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

# Preprocess Feature Columns
As you can see, there are several non-numeric columns that need to be converted! Many of them are simply yes/no, e.g. internet. These can be reasonably converted into 1/0 (binary) values.

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply yes/no, e.g. internet. These can be reasonably converted into 1/0 (binary) values.
Other columns, like Mjob and Fjob, have more than two values, and are known as categorical variables. The recommended way to handle such a column is to create as many columns as possible values (e.g. Fjob_teacher, Fjob_other, Fjob_services, etc.), and assign a 1 to one of them and 0 to all others.

These generated columns are sometimes called dummy variables, and we will use the pandas.get_dummies() function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.


In [47]:
def preprocess_features(X):
# preprocess the student data and convert into non-numeric binary data into binary(0/1).Convert categorail variables into dummy 
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)
    
    # Investigate each feature column for the data
    for col,col_data in X.iteritems() :
        # if datatype non-numeric binary
        if col_data.dtype == object:
            col_data = col_data.replace(['yes','no'],[1,0])
            
        # If data type is categorical, convert to dummy variables    
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data= pd.get_dummies(col_data,prefix=col)
            
        # Collect the revised columns
        output = output.join(col_data)   
    return output

X_all = preprocess_features(X_all)

print("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all categorical features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. 
In the following code cell below, you will need to implement the following:
Randomly shuffle and split the data (X_all, y_all) into training and testing subsets.
Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
Set a random_state for the function(s) you use, if provided.
Store the results in X_train, X_test, y_train, and y_test.

In [54]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
from sklearn.utils import shuffle

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, Y_all, train_size=num_train, test_size=num_test)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in scikit-learn. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F1 score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F1 score on the training set, and F1 score on the testing set.
The following supervised learning models are currently available in scikit-learn that you may choose from:

Gaussian Naive Bayes (GaussianNB)
Decision Trees
Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
K-Nearest Neighbors (KNeighbors)
Stochastic Gradient Descent (SGDC)
Support Vector Machines (SVM)
Logistic Regression

Model Application
List three supervised learning models that are appropriate for this problem. For each model chosen
Describe one real-world application in industry where the model can be applied. (You may need to do a small bit of research for this — give references!) 
What are the strengths of the model; when does it perform well? 
What are the weaknesses of the model; when does it perform poorly?
What makes this model a good candidate for the problem, given what you know about the data?

Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
train_classifier - takes as input a classifier and training data and fits the classifier to the data.
predict_labels - takes as input a fit classifier, features, and a target labeling and makes predictions using the F1 score.
train_predict - takes as input a classifier, and the training and testing data, and performs train_clasifier and predict_labels.
This function will report the F1 score for both the training and testing data separately. 

In [55]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))
    print("\n")

# Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the train_predict function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
Import the three supervised learning models you've discussed in the previous section.
Initialize the three models and store them in clf_A, clf_B, and clf_C.
Use a random_state for each model you use, if provided.
Note: Use the default settings for each model — you will tune one specific model in a later section.
Create the different training set sizes to be used to train each model.
Do not reshuffle and resplit the data! The new training points should be drawn from X_train and y_train.
Fit each model with each training set size and make predictions on the test set (9 in total).
Note: Three tables are provided after the following code cell which can be used to store your results.

In [56]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

# TODO: Initialize the three models
clf_A = RandomForestClassifier(random_state=0)
clf_B = GaussianNB()
clf_C = LogisticRegression(random_state=0)

# TODO: Set up the training set sizes
# Previously shuffled
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C]:
    for j in [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]:
        train_predict(clf, j[0], j[1], X_test, y_test)

C:\Users\573380\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.7988 seconds
Made predictions in 0.0117 seconds.
F1 score for training set: 0.9925.
Made predictions in 0.0122 seconds.
F1 score for test set: 0.7519.


Training a RandomForestClassifier using a training set size of 200. . .
Trained model in 0.0160 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.9963.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7164.


Training a RandomForestClassifier using a training set size of 300. . .
Trained model in 0.0621 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.9927.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7518.


Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8092.
Made predictions in 0.0060 seconds.
F1 score for test set: 0.6720.


Training a GaussianNB using a traini

In [57]:
# Models 4 - 7 for general comparison

# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier


# TODO: Initialize the three models
clf_A = DecisionTreeClassifier(random_state=0)
clf_B = SGDClassifier(random_state=0)
clf_C = SVC(random_state=0)
clf_D = KNeighborsClassifier()

# TODO: Set up the training set sizes
# Previously shuffled
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C, clf_D]:
    for j in [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]:
        train_predict(clf, j[0], j[1], X_test, y_test)

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7727.


Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0010 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.6942.


Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0030 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7154.


Training a SGDClassifier using a training set size of 100. . .
Trained model in 0.1308 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8289.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7785.


Training a SGDClassifier using a 

C:\Users\573380\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)



Trained model in 0.0140 seconds
Made predictions in 0.0160 seconds.
F1 score for training set: 0.8725.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7123.


Training a SVC using a training set size of 200. . .
Trained model in 0.0030 seconds
Made predictions in 0.0020 seconds.
F1 score for training set: 0.8904.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7234.


Training a SVC using a training set size of 300. . .
Trained model in 0.0259 seconds
Made predictions in 0.0040 seconds.
F1 score for training set: 0.8726.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7448.


Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0389 seconds
Made predictions in 0.3438 seconds.
F1 score for training set: 0.8750.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.7482.


Training a KNeighborsClassifier using a training set size of 200. . .
Trained model in 0.0280 seconds
Made predictions in 0.0039 second

Tabular Results
Edit the cell below to see how a table can be designed in Markdown. You can record your results from above in the tables provided.

** Classifer 1 - Random Forest**  

| Training Set Size | Training Time (s) | Prediction Time (s) (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0102                   |     0.0009                   |      0.9922            |         **0.7206**        |
| 200               |        0.0094          |            0.0008            |          0.9962        |       0.6977          |
| 300               |           0.0107              |         0.0012               |        0.9951          |    0.6721      |

* High F1 score for train (1.0000) vs lower F1 score for test (0.6667 to 0.7460) indicates there is overfitting and that the model is not generalising well.
* F1 test score decreases as training set size increases, again suggesting that there is overfitting.
* Training time is high. (about 10x that of GaussianNB, KNeighborsClassifier)

** Classifer 2 - GaussianNB**  

| Training Set Size | Training Time (s)| Prediction Time (test) (s)| F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |          0.0009               |     0.0004                   |     0.8392             |     **0.7591**            |
| 200               |     0.0007                    |   0.0002                     |   0.8309               |     0.7424     |
| 300               |     0.0011             |    0.0003                    |    0.8099              |    0.7463             |

** Classifer 3 - Logistic Regression**  

| Training Set Size | Training Time (s)| Prediction Time (test) (s)| F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |     0.0027                    |        0.0001                |    0.8872              |   0.7328              |
| 200               |     0.0017             |          0.0002              |          0.8489        |          0.7612       |
| 300               |        0.0026                 |      0.0001                  |    0.8337              |     **0.7883**     |

It's doing surprisingly well.

** Classifer 4 - Support Vector Machines SVC**  

| Training Set Size | Training Time (s)| Prediction Time (test) (s)| F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0038                 |    0.0007                    |            0.8671      |    0.7483             |
| 200               |     0.0033             |     0.0013                   |   0.8800               |     0.7724            |
| 300               |        0.0053                 |      0.0013                  |     0.8793             |     **0.7808**     |

* This also did surprisingly well compared to expectations. I thought SVM wasn't great with data that had many features. Maybe this dataset doesn't actually have that many features (vs 50,000 features for some text learning applications).
* Prediction time linear with number of things to predict for training set sizes 200,300.

** Classifer 5 - KNeighborsClassifier**  

| Training Set Size | Training Time (s)| Prediction Time (test) (s)| F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |    0.0006                     |    0.0012                    |    0.8345              |     0.7023            |
| 200               |     0.0006             |      0.0014                  |       0.8502           |   0.7121              |
| 300               |      0.0007                   |    0.0019                    |   0.8731               |     **0.7556**     |


** Classifer 6 - Decision Trees**  

| Training Set Size | Training Time (s)| Prediction Time (test) (s)| F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |           0.0009              |    0.0004                    |   1.0000               |     0.6667            |
| 200               |     0.0013             |        0.0001                |   1.0000               |    **0.7460**             |
| 300               |     0.0016                    |     0.0002                   |      1.0000            |     0.7424     |

* High F1 score for train (1.0000) vs lower F1 score for test (0.6667 to 0.7460) indicates there is overfitting and that the model is not generalising well.
* F1 score peaks at 200 training points and decreases slightly at 300 training points, suggesting there is overfitting at 300 training points.

** Classifer 7 - Stochastic Gradient Descent**  

| Training Set Size | Training Time (s)| Prediction Time (test) (s)| F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0091                |     0.0008                   |    0.7832              |     0.7586            |
| 200               |     0.0010             |    0.0002                    |      0.5027            |     0.3902            |
| 300               |       0.0010                  |         0.0002               |  0.5981                |     0.4946     |



# Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

# Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

# *Answer: **

I chose **Logistic Regression**.

1. **Performance (important)**: Logistic Regression had the **highest F1 score**. 
    * F1 score is a combined measure of (the harmonic mean of) precision and recall.
        - Precision is X and 
        - Recall is Y.
    * The other comparable model is SVC. Logistic Regression and SVC have maximum F1 scores of **0.7883** and 0.7808 respectively, with both attaining F1 max at 300 training points. They are significantly higher than the third highest F1 score at 0.7591 for GaussianNB.

2. **Cost** (measured by training and prediction times):
    * Out of the two models with similar F1 score, Logistic Regression has the **shorter training and prediction time** at < 50% that af SVC's time (**0.0026s to train, 0.0001s to predict** vs SVC 0.0053s to train and 0.0013 to predict). 
    * The training time is not too high and the prediction time is extremely low at 0.0001s.
    * Since minimising computational cost is a concern, Logistic Regression seems like a good choice.

3. **Available data**
    * This is taken into account by the F1 scores used to assess the models. Three models - KNeighborsClassifier, SVC and Logistic Regression - have F1 scores that increase as the number of training datapoints increases. These models may perform even better if we had, say, 500 training data points. One of SVC or KNeighborsClassifier might outperform Logistic Regression. But we only have 300 data points, so we need to make choices based on the F1 scores in the tables.

**Note 1: Backup in case even Logistic Regression is too computationally expensive: GaussianNB**

If the computational cost is considered too great, we choose KNeighborsClassifier, which has a training time of 0.0009s, a prediction time of 0.0004s and an F1 score of 0.7591 on 100 training points, the third highest F1 score and the shortest (training time + prediction time).
* It is only trained on 100 datapoints which makes it feel a bit dodgy. I considered choosing KNeighborsClassifier which has the fourth highest F1 score (max at 300 training points) and a shorter training time, but its prediction time of 0.0019s is quite high, making computational cost high.

**Note 2: This may not be the optimal model because we did not tune any parameters.**
* The default parameters for e.g. Decision Trees may just be really bad for this example.
* If we wanted to choose the best model, we should compare versions of the models with tuned parameters.